
## 🔹 Task 1: Managing Conversation History with Summarization



In [1]:

from collections import deque

class ConversationManager:
    def __init__(self, max_turns=None, max_chars=None, summarize_every=3):
        self.history = []
        self.max_turns = max_turns
        self.max_chars = max_chars
        self.summarize_every = summarize_every
        self.run_count = 0

    def add_message(self, role, content):
        self.history.append({"role": role, "content": content})
        self.run_count += 1
        self._truncate_history()
        if self.run_count % self.summarize_every == 0:
            self._summarize_history()

    def _truncate_history(self):
        if self.max_turns:
            self.history = self.history[-self.max_turns:]
        if self.max_chars:
            total_chars = 0
            truncated = []
            for msg in reversed(self.history):
                if total_chars + len(msg["content"]) <= self.max_chars:
                    truncated.insert(0, msg)
                    total_chars += len(msg["content"])
                else:
                    break
            self.history = truncated

    def _summarize_history(self):
        summary = []
        for msg in self.history:
            snippet = " ".join(msg["content"].split()[:5]) + ("..." if len(msg["content"].split()) > 5 else "")
            summary.append(f"{msg['role']}: {snippet}")
        summarized_text = " | ".join(summary)
        self.history = [{"role": "system", "content": f"Summary so far: {summarized_text}"}]

    def get_history(self):
        return self.history


# Demo Task 1
cm = ConversationManager(max_turns=4, max_chars=150, summarize_every=3)

sample_conversation = [
    ("user", "Hello, I am looking for movie suggestions."),
    ("assistant", "Sure! What genre are you interested in?"),
    ("user", "I like action and thriller movies."),
    ("assistant", "Great! Inception and John Wick could be good choices."),
    ("user", "Thanks, I’ll watch Inception."),
    ("assistant", "You're welcome! Enjoy the movie.")
]

for role, msg in sample_conversation:
    cm.add_message(role, msg)
    print("Conversation history after this message:")
    print(cm.get_history())
    print("-"*50)


Conversation history after this message:
[{'role': 'user', 'content': 'Hello, I am looking for movie suggestions.'}]
--------------------------------------------------
Conversation history after this message:
[{'role': 'user', 'content': 'Hello, I am looking for movie suggestions.'}, {'role': 'assistant', 'content': 'Sure! What genre are you interested in?'}]
--------------------------------------------------
Conversation history after this message:
[{'role': 'system', 'content': 'Summary so far: user: Hello, I am looking for... | assistant: Sure! What genre are you... | user: I like action and thriller...'}]
--------------------------------------------------
Conversation history after this message:
[{'role': 'assistant', 'content': 'Great! Inception and John Wick could be good choices.'}]
--------------------------------------------------
Conversation history after this message:
[{'role': 'assistant', 'content': 'Great! Inception and John Wick could be good choices.'}, {'role': 'user'


## 🔹 Task 2: JSON Schema Classification & Information Extraction




In [2]:

import os
from openai import OpenAI

os.environ["GROQ_API_KEY"] = "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"

client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)


In [3]:

import json

functions = [
    {
        "name": "extract_user_info",
        "description": "Extract user details from chat",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "email": {"type": "string"},
                "phone": {"type": "string"},
                "location": {"type": "string"},
                "age": {"type": "integer"}
            },
            "required": ["name", "email", "phone", "location", "age"]
        },
    }
]

# Sample chats
chats = [
    "Hi, I am Piyush Borney, 22 years old, from Nagpur. My email is piyush@example.com and my phone is 9876543210.",
    "Hello, I am Sarah, 30, living in New York. You can reach me at sarah123@gmail.com or call at 1234567890.",
    "Hey, name's Arjun, 25, based in Bangalore. Contact: arjun@workmail.com, phone 8887776665."
]

for chat in chats:
    response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",   
    messages=[{"role": "user", "content": chat}],
    functions=functions,
    function_call={"name": "extract_user_info"}   
)

    structured = json.loads(response.choices[0].message.function_call.arguments)
    print("Chat:", chat)
    print("Extracted JSON:", structured)
    print("-"*60)


Chat: Hi, I am Piyush Borney, 22 years old, from Nagpur. My email is piyush@example.com and my phone is 9876543210.
Extracted JSON: {'age': 22, 'email': 'piyush@example.com', 'location': 'Nagpur', 'name': 'Piyush Borney', 'phone': '9876543210'}
------------------------------------------------------------
Chat: Hello, I am Sarah, 30, living in New York. You can reach me at sarah123@gmail.com or call at 1234567890.
Extracted JSON: {'age': 30, 'email': 'sarah123@gmail.com', 'location': 'New York', 'name': 'Sarah', 'phone': '1234567890'}
------------------------------------------------------------
Chat: Hey, name's Arjun, 25, based in Bangalore. Contact: arjun@workmail.com, phone 8887776665.
Extracted JSON: {'age': 25, 'email': 'arjun@workmail.com', 'location': 'Bangalore', 'name': 'Arjun', 'phone': '8887776665'}
------------------------------------------------------------
